<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph(1)_Basics_with_ReAct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph (1): 基礎とReActエージェントの構築

LangChain の基礎コースに続き、本ノートブックからは **LangGraph** を用いたエージェント構築を扱います。

LangChain が一方向の処理連鎖（DAG）を主としていたのに対し、LangGraph はループを含む循環グラフ構造を定義可能です。これにより、LLM が自身の出力やツール実行結果に基づいて次の行動を決定する、自律的なエージェント機能（ReAct パターンなど）の実装が容易になります。

### 学習目標
* **State Management:** ノード間で共有される状態（State）の定義と更新メカニズムの理解。
* **Graph Construction:** ノードとエッジによるグラフ構造の設計と可視化。
* **ReAct Agent:** 条件付きエッジとループ構造を用いた、自己回帰的な検索エージェントの実装。

### 1. 環境構築とAPI設定

`langgraph` および関連ライブラリをインストールします。グラフ構造の可視化には `grandalf` を使用します。

In [ ]:
!pip install -qU langgraph langchain-openai langchain-community tavily-python termcolor grandalf

import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from termcolor import cprint

# APIキーの設定
try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
    cprint("API Keys loaded.", "green")
except Exception:
    cprint("Error: Please set OPENROUTER_API_KEY and TAVILY_API_KEY in secrets.", "red")

# LLMの初期化
# model は deepseek/deepseek-chat-v3-0324 を使用
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    model="deepseek/deepseek-chat-v3-0324",
    temperature=0
)

## 2. LangGraph の基本構造

LangGraph を理解する上で重要なのが、システム全体で共有される「状態（ステート, State）」という概念です。

これまでの手法では、ある処理の結果を次の処理へバケツリレーのように直接渡していました。一方、LangGraph では「現在の状況」を記録する場所を一つ用意します。それぞれの処理担当者（ノード）は、その共有された記録を見て自分の仕事を行い、結果をまたその記録場所に書き込みます。これにより、複雑なデータのやり取りや記憶の管理が容易になります。

### 状態（ステート）の定義

まず、エージェントがどのような情報を記憶しておくべきかを定義します。ここでは、ユーザーとの会話の履歴をリスト形式で保持するように設定します。また、新しい発言があった場合のルールとして、過去の履歴を消去して上書きするのではなく、既存の履歴の後ろに新しい内容を順番に追加していくように指定します。

`TypedDict` を使用して State のスキーマを定義します。ここでは会話履歴を保持する `messages` フィールドを作成します。
`Annotated` と `add_messages` リデューサを使用することで、ノードからの出力が既存リストに追記（append）される挙動を定義します。

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    # メッセージのリストを保持するState
    # add_messages は、新しいメッセージが返されたときに既存のリストに追加する処理（Reducer）です
    messages: Annotated[list, add_messages]

print("State schema defined.")

### 処理担当（ノード）の実装

「ノード」とは、グラフの中で実際に特定の作業を行う処理単位のことです。各ノードは現在の状況（ステート）を受け取り、自分の仕事の結果として更新分を返します。

ここでは、AIモデルにこれまでの会話履歴を読ませ、適切な返答を生成させる処理を作成します。AIが生成した返答は、先ほど定義した共有の記録場所に追加されることになります。

In [ ]:
def chatbot_node(state: State):
    """
    現在の会話履歴を用いてLLMに応答を生成させるノード
    """
    # 1. Stateから現在のメッセージ履歴を取得
    messages = state["messages"]

    # 2. LLMを実行 (invoke)
    response = llm.invoke(messages)

    # 3. Stateの更新差分を返却
    # ここで返したメッセージが、Stateのmessagesリストの末尾に追加されます
    return {"messages": [response]}

### グラフの構築とコンパイル

定義した処理単位（ノード）を配置し、それらを線（エッジ）で繋いで全体の流れを作ります。

ここでは「開始地点」から「AIの処理」へ進み、処理が終わったら「終了地点」へ向かうという、最も単純な一本道の流れを構築します。最後に、この設計図をプログラムとして実行可能な形式に変換します。

`StateGraph` クラスを用いてグラフを構築します。
1. `add_node`: 定義した関数をノードとして登録。
2. `add_edge`: ノード間の遷移を定義（START -> chatbot -> END）。
3. `compile`: 実行可能な `CompiledGraph` オブジェクトを生成。

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# グラフビルダーの初期化
graph_builder = StateGraph(State)

# ノードとエッジの追加
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_edge(START, "chatbot") # 開始 -> chatbot
graph_builder.add_edge("chatbot", END)   # chatbot -> 終了

# コンパイル (実行可能オブジェクトの生成)
graph = graph_builder.compile()

# グラフ構造の可視化 (Mermaid)
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization failed: {e}")

### 実行確認

`graph.stream` を使用して、初期状態を入力し、グラフを実行します。初期状態として最初の会話文を与えると、グラフの流れに従って各ステップでのノードの出力がストリームされます。

In [ ]:
initial_state = {"messages": [("user", "LangGraphの学習を開始します。")]}

# streamメソッドでグラフを実行
# event には各ノードの出力が含まれます
for event in graph.stream(initial_state):
    for node_name, values in event.items():
        print(f"--- Node: {node_name} ---")

        # values["messages"] はそのノードが生成した新しいメッセージのリスト
        last_msg = values["messages"][-1]

        # LLMの応答を表示
        print(f"Output: {last_msg.content}")

## 3. ReAct エージェントの実装 (循環グラフ)

LLM がツール実行の必要性を判断し、実行結果に基づいて再推論を行う ReAct (Reasoning and Acting) パターンを実装します。
単純な一本道ではなく、条件によって進む道を変える仕組みと、納得いくまで処理を繰り返すためのループ構造を取り入れます。この構造には、条件に応じて遷移先を変える **Conditional Edge** と、処理を戻す **Cycle** が必要です。これにより、AIは検索結果を見て「情報が足りないからもう一度調べよう」といった試行錯誤ができるようになります。

### ツールの設定

LLMが外部の情報を調べるために使う検索機能を準備します。LLM自身はインターネットに直接接続できないため、検索専用の道具を持たせ、その使い方を認識させます。ここでは Tavily Search API をツールとして定義し、LLM にバインドします。

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 検索ツールの初期化 (最大3件取得)
tool = TavilySearchResults(max_results=3)
tools = [tool]

# LLMにツールをバインド
# これにより、LLMは回答の代わりに「ツール実行リクエスト (tool_calls)」を出力できるようになります
llm_with_tools = llm.bind_tools(tools)

### ノードと条件付き分岐の定義

ここでは、「思考を担当するノード」と「道具の使用を担当するノード」の二つを用意します。

重要なのは、思考担当ノードの後に設置する分岐点です。「AIがツールを使いたいと言ったかどうか」を判断基準にして、自動的に進む道が切り替わるように設定します。さらに、ツールを使った後は、その結果を持って再び思考担当ノードに戻るように道を繋ぎます。


* **agent:** LLM が思考し、応答またはツール呼び出し（tool_calls）を生成します。
* **tool:** `tool_calls` を解析し、実際にツールを実行します。LangGraph の `ToolNode` を使用します。
* **conditional edge:** LLM の出力に `tool_calls` が含まれる場合は Tool Node へ、そうでなければ終了 (END) へ遷移します。

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

def agent_node(state: State):
    """
    現在のStateに基づいてLLMを実行し、次のアクション（回答 or ツール呼び出し）を決定するノード
    """
    messages = state["messages"]

    # ツール情報付きのLLMを実行
    # 結果には通常のテキスト、または tool_calls が含まれる
    response = llm_with_tools.invoke(messages)

    # 結果をStateに追加
    return {"messages": [response]}

# ツール実行用ノード (LangGraph提供)
# 直前のメッセージに含まれる tool_calls を自動的に実行し、結果を ToolMessage として返します
tool_node = ToolNode(tools)

# --- グラフ構築 ---
agent_builder = StateGraph(State)

# ノードの追加
agent_builder.add_node("agent", agent_node)
agent_builder.add_node("tools", tool_node)

# エッジの定義
agent_builder.add_edge(START, "agent")

# 条件付きエッジ (Conditional Edge)
# "agent" ノードの出力後に実行され、次の遷移先を動的に決定します
# tools_condition は以下のロジックを持ちます：
# - 直前のメッセージに tool_calls がある -> "tools" へ遷移
# - ない -> END へ遷移
agent_builder.add_conditional_edges(
    "agent",
    tools_condition,
    {"tools": "tools", "__end__": END}
)

# 循環エッジ (Cycle)
# ツール実行後は必ず agent に戻り、検索結果を踏まえて再考させます
agent_builder.add_edge("tools", "agent")

agent_graph = agent_builder.compile()

display(Image(agent_graph.get_graph().draw_mermaid_png()))

## 4. エージェントの実行

複雑なクエリを入力し、LLM がどのように判断（出力）し、ツールがどう実行されたかを確認します。
中間出力を詳細に表示することで、グラフ内のデータの流れを追跡します。

In [ ]:
from langchain_core.messages import HumanMessage
from termcolor import colored

def print_stream(graph, query):
    print(f"Query: {query}\n")
    inputs = {"messages": [HumanMessage(content=query)]}

    # グラフをステップ実行
    for event in graph.stream(inputs):
        for node_name, values in event.items():
            print(colored(f"=== Node: {node_name} ===", "blue", attrs=["bold"]))

            # このノードで生成されたメッセージを取得
            messages = values["messages"]
            for msg in messages:
                # 1. LLMがツール呼び出しを決定した場合
                if hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
                    print(colored("decision: Tool Call Required", "magenta"))
                    for tc in msg.tool_calls:
                        print(f"  - Tool: {tc['name']}")
                        print(f"  - Args: {tc['args']}")

                # 2. ツール実行結果の場合
                elif msg.type == "tool":
                    print(colored("Action: Tool Executed", "green"))
                    # 結果が長い場合は切り詰めて表示
                    content_preview = msg.content[:200] + "..." if len(msg.content) > 200 else msg.content
                    print(f"  - Result: {content_preview}")

                # 3. 最終的な回答の場合
                else:
                    print(colored("Decision: Final Answer", "cyan"))
                    print(msg.content)
            print("\n")

# 実行
query = "LangGraphの主な機能と、LangChainとのアーキテクチャ上の違いを解説してください。"
print_stream(agent_graph, query)

### まとめ

本セクションでは LangGraph を用いて、自分で考えて道具を使うエージェントを作成しました。

1.  **状態の共有:** 処理の間でデータを受け渡すための「記録場所」を定義しました。
2.  **ループ構造:** 道具を使った後に再び思考に戻ることで、試行錯誤を可能にしました。
3.  **条件分岐:** 状況に応じて、道具を使うか回答するかを自動で判断させました。

次回は、会話の内容を長期的に覚えておくための記憶の仕組み（永続化）と、自分の回答を見直して修正する機能の実装に進みます。

## 5. 【演習】 自作ツールの追加

Tavily 検索以外のツールをエージェントに追加してみましょう。
ここでは、入力されたテキストの文字数をカウントする単純なツール `count_chars` を作成し、エージェントに組み込みます。

**課題:**
以下のコードの `■■■` を埋めて完成させ、エージェントが「検索」と「文字数カウント」の2つのツールを適切に使い分けるか確認してください。

In [ ]:
from langchain_core.tools import tool

# 1. 自作ツールの定義
# 関数をツールとして LangChain に認識させるためのデコレータ
@■■■
def count_chars(text: str) -> int:
    """
    指定されたテキストの文字数をカウントします。空白や改行も1文字として数えます。
    """
    # 引数 text の長さを計算して返す
    return ■■■

# 2. ツールリストの更新
# 既存の TavilySearchResults (max_results=1) と、上で作った count_chars をリストにする
new_tools = [TavilySearchResults(max_results=1), ■■■]

# 3. 新しいLLMとノードの作成
# 定義済みの llm に対して、作成したツールリストをバインド (紐付け) する
llm_with_new_tools = llm.■■■(new_tools)

# ツールを実行するためのノード (ToolNode) を作成する
new_tool_node = ■■■(new_tools)

# エージェントノード (思考担当) の定義
def new_agent_node(state: State):
    # State の辞書から "messages" キーの値を取得する
    messages = state[■■■]

    # ツール情報を持った LLM を実行 (invoke) する
    response = llm_with_new_tools.■■■(messages)

    # 結果を State の形式 (辞書) で返す。キーは "messages"、値はリストにする
    return {■■■: [response]}

# 4. 新しいグラフの構築
# StateGraph を State クラスを使って初期化する
new_builder = ■■■(State)

# ノードを登録する ("agent" という名前で new_agent_node を登録)
new_builder.■■■("agent", new_agent_node)
# ノードを登録する ("tools" という名前で new_tool_node を登録)
new_builder.■■■("tools", new_tool_node)

# 開始地点 (START) から agent ノードへのエッジ (線) を追加する
new_builder.■■■(START, "agent")

# 条件付きエッジ (Conditional Edge) を追加する
# agent の出力に基づき、ツールを使うなら "tools" へ、そうでなければ終了 (END) へ分岐させる
# 分岐判定ロジックには既定の tools_condition を使用する
new_builder.■■■(
    "agent",
    ■■■,
    {"tools": "tools", "__end__": END}
)

# 循環エッジ: ツール実行後は必ず agent に戻るようにエッジを追加する
new_builder.■■■("tools", "agent")

# グラフをコンパイルして実行可能な状態にする
new_graph = new_builder.■■■()

# 5. 実行確認 (ここは記述済み)
# 検索が必要な質問と、計算が必要な質問を組み合わせたクエリ
test_query = "「LangGraph」という単語の文字数を数えてください。また、LangGraphの最新バージョンについて検索して教えて。"
print_stream(new_graph, test_query)

### まとめ

本セクションでは LangGraph によるエージェント構築の基礎を実装しました。

1.  **StateGraph:** 共有状態を持つグラフ構造の定義。
2.  **Cyclic Execution:** `tools` から `agent` へのエッジによる、自律的なループ処理の実現。
3.  **Conditional Logic:** `tools_condition` による動的な制御フローの実装。

次回は、MemorySaver を用いた会話履歴の永続化 (Persistence) と、Reflection パターンの実装に進みます。